<a href="https://colab.research.google.com/github/dldowning/Fall2022-5222/blob/main/5222_Tweet_Analysis_Revision_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [286]:
from statistics import mean
import math
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))


In [287]:
#read file names into list - will later us to iterate gathering data 
my_file = open("/content/drive/MyDrive/5290/5290.Data/subreddit_files.txt", "r")
data = my_file.read()

data_into_list = data.split("\n")
print(data_into_list)
my_file.close()

['3DS.tsv', '4chan.tsv', '2007scape.tsv', 'ACTrade.tsv', 'amiugly.tsv', 'BabyBumps.tsv', 'baseball.tsv', 'canada.tsv', 'CasualConversation.tsv', 'DarknetMarkets.tsv', 'darksouls.tsv', 'elderscrollsonline.tsv', 'Eve.tsv', 'Fallout.tsv', 'fantasyfootball.tsv', 'GameDeals.tsv', 'gamegrumps.tsv', 'halo.tsv', 'Homebrewing.tsv', 'IAmA.tsv', 'india.tsv', 'jailbreak.tsv', 'Jokes.tsv', 'KerbalSpaceProgram.tsv', 'Keto.tsv', 'leagueoflegends.tsv', 'Libertarian.tsv', 'magicTCG.tsv', 'MakeupAddiction.tsv', 'Naruto.tsv', 'nba.tsv', 'occulus.tsv', 'OkCupid.tsv', 'Parenting.tsv', 'pathofexile.tsv', 'raisedbynarcissists.tsv', 'Random_Acts_Of_Amazon.tsv', 'science.tsv', 'Seattle.tsv', 'TalesFromRetail.tsv', 'talesfromtechsupport.tsv', 'ultrahardcore.tsv', 'videos.tsv', 'Warthunder.tsv', 'histfreq2000.tsv', 'histadj2000.tsv']


In [288]:
#Use file names to create dataframe names 

subreddit_dataframes = []            # List to hold subreddit dataframes 

for i in range(0,len(data_into_list)):   # For range in length of the # of subreddits we will use (44)    
  x = data_into_list[i]                  # at index i of data_into_list assign the str value to x
  dataframe_name = 'df_'+ x[:-4]          # with the assigned x value, append 'df_' to the front of te string and remove the last 4 characters ".tsv". Leaves "df_4chan"

  foo1 = dataframe_name
  exec(foo1 + " = subreddit_dataframes.append(pd.read_csv(\"/content/drive/MyDrive/5290/5290.Data/subreddits/" + x +"\",sep = '\t', header=None)) ")    # Convert the string stored in dataframe_name to a variable name to hold the respective dataframe 
                                                                                                                                          # https://stackoverflow.com/questions/11553721/using-a-string-variable-as-a-variable-name
  subreddit_dataframes[i].columns = ['Word', 'Sentiment Score', 'drop']            # Create clumn names in each respective dataframe 
  subreddit_dataframes[i] = subreddit_dataframes[i].drop('drop', axis=1)                    # Drop unwanted column

# One Function To Create Train and Test Dataframes

In [348]:
# Allow about 16min for runtime

def feature_extraction(data_location, label_location):
  df = pd.read_csv(data_location, header=None)
  df.columns = ['TWEET']

  df["Tweet Tokens"] = np.nan
  df["Count: Words in + Lexicon"] = np.nan
  df["Count: Words in - Lexicon"] = np.nan
  df["Contain The word NO? "] = np.nan
  df["Count: Nouns"] = np.nan
  df["Ratio: Unique Words-Total Words"] = np.nan
  df["Ratio: Stop Words-Total Words"] = np.nan
  df["Count: Adjectives in Tweet"] = np.nan
  df["Log: Tweet word count"] = np.nan
  df["Log: Length of Longest Word in Tweet"] = np.nan
  df["Log: Count of Words with 5+ Characters"] = np.nan

  # ADD LABELS COLUMN TO DF 
  labels = pd.read_csv(label_location, sep="\n", header=None)
  df = pd.concat([df,labels], axis = 1)
  df.rename(columns = {0:'Labels'}, inplace = True)

  # DROP NEUTRAL LABELS FROM DF
  df.drop(df.loc[df['Labels']==1].index, inplace=True)
  df = df.reset_index(drop=True)

  # CHANGE ALL 2 LABEL VALUES TO 1 
  for i in range(0, len(df.index)):
    if df['Labels'].values[i] == 2:
      df.at[i,'Labels'] = 1

  # CLEAN TWEETS
  pattern_a = r'[^A-Za-z0-9]+'
  pattern_b = r'\b\w{1,1}\b'
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)

  for i in range(0, len(df.index)):                       
    df['TWEET'].values[i] = df['TWEET'].values[i].lower()
    df['TWEET'].values[i] = df['TWEET'].values[i].replace('@user', '')
    df['TWEET'].values[i] = re.sub(pattern_a, ' ', df['TWEET'].values[i])
    df['TWEET'].values[i] = re.sub(pattern_b, '', df['TWEET'].values[i])
    df['TWEET'].values[i] = re.sub(emoji_pattern, '', df['TWEET'].values[i])

  #TOKENIZE TWEETS
  df['Tweet Tokens'] = df['Tweet Tokens'].astype('object')
  for i in range(0, len(df.index)):
    
    tokens = df['TWEET'].values[i].split()
    df.at[i, 'Tweet Tokens'] = tokens

  #Create Positive and Negative Lexicons
  pos_lexicon = []
  neg_lexicon = []

  for i in range(0, len(subreddit_dataframes)):               
    for j in range(0, len(subreddit_dataframes[i].index)):    

      if subreddit_dataframes[i]['Sentiment Score'].values[j] >= 0:         
        pos_lexicon.append(subreddit_dataframes[i]['Word'].values[j])        
      
      if subreddit_dataframes[i]['Sentiment Score'].values[j] < 0:         
        neg_lexicon.append(subreddit_dataframes[i]['Word'].values[j])        


  pos_lexicon = [*set(pos_lexicon)]   # Remove duplicate values from + lexicon
  neg_lexicon = [*set(neg_lexicon)]   # Remove duplicate values from - lexicon


  #Handle duplicate words in postitive and negative lexicon
  same_wrds = set(pos_lexicon).intersection(neg_lexicon)      #get set of all words that appear in both psoitive and Negative Lexicon
                                                              #https://stackoverflow.com/questions/1388818/how-can-i-compare-two-lists-in-python-and-return-matches

  word_vals_dict = dict.fromkeys(same_wrds, 0)                # Create a dictionary to hold of all words found in positive and negative lexicons     
  sentiment_vals2sum = []

  for k in range(0, len(same_wrds)):    # In the set of words identified in positive and negative lexicon
  
    i = same_wrds.pop()                 # i will return one word from the set, then the following with each iteration
    same_wrds.add(i)                    # https://stackoverflow.com/questions/59825/how-to-retrieve-an-element-from-a-set-without-removing-it

    sentiment_vals2sum = []             # Will store the Sentiment Scores collected across data frames 

    for j in range(0, len(subreddit_dataframes)):                    # in range of dataframes(44)
      is_wrd_there = i in subreddit_dataframes[j]['Word'].unique()   # Return true or false. True if desired word 'i' is in the data frame being checked false if not (ls_df_names[0], ls_df_names[1],...)
      if is_wrd_there is True:                              # If true... 

        mask1 = subreddit_dataframes[j]['Word'].values == i                  # Get the sentiment value of the word from its dataframe 
                                                                    # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
                                                      
        sentiment_vals2sum.append(subreddit_dataframes[j][mask1].iat[0,1])   # append sentiment value to list where they are stored eg. sentiment_vals2sum 
        num_avg = mean(sentiment_vals2sum)                          # Take the mean of the sentiment scores collected in the list above(these are all sentiment scores for one word collected across dataframes where the word was found 'True')
        word_vals_dict[i] = num_avg  

  # Remove words that now have a clear positive or negative classification
  for w in word_vals_dict.items():        # .items() returns a tuple of (word, score). See Cell above for all words: scores in dict
    if w[1] >= 0:                         # if w[1] (the score) is greater than 0...
      neg_lexicon.remove(w[0])            # remove it from the negative lexicon
    if w[1] < 0:                          # If w[1] (the score) is less than 0...
      pos_lexicon.remove(w[0])            # remove the word from the positive lexicon

  # COUNT POSITIVE AND NEGATIVE WORDS
  neg_lex_set = set(neg_lexicon)
  pos_lex_set = set(pos_lexicon)

  for i in range(0, len(df.index)):
    x = set(df['Tweet Tokens'][i])
    df['Count: Words in + Lexicon'].values[i] = len(x.intersection(pos_lex_set))
    df['Count: Words in - Lexicon'].values[i] = len(x.intersection(neg_lex_set))

  # DOES TWEET CONTAIN NO
  for i in range(0, len(df.index)):                       # In the range 0 to length of the tweets dataframe         # Tokenize and lowercase tweets 
    if 'no' in df['Tweet Tokens'][i]:                                    # If no is in tweet dataframe value is 1 if not value is zero
      df['Contain The word NO? '].values[i] = 1
    else: 
      df['Contain The word NO? '].values[i] = 0

  # COUNT NOUNS IN TWEET
  for i in range(0, len(df.index)):
    tokens = df['Tweet Tokens'][i]
    tokens = [w for w in tokens if not w in stop_words]
    tagged = nltk.pos_tag(tokens)

    noun_definitions = ['NN', 'NNS', 'NNP', 'NNPS']
    count = 0

    for j in range(0, len(tagged)):
      if tagged[j][1] in noun_definitions:
        count += 1
    
    df['Count: Nouns'].values[i] = count    

  # RATIO: UNIQUE TO TOTAL WORDS
  for i in range(0, len(df.index)):                       
    tokens = df['Tweet Tokens'][i]
    x = np.array(tokens)
    ratio = len(np.unique(x)) / len(tokens)
    df['Ratio: Unique Words-Total Words'].values[i] = ratio    

  # STOP WORDS TO TOTAL WORDS
  for i in range(0, len(df.index)):                       
    tokens = df['Tweet Tokens'][i]
    x = np.array(tokens)
    stop_wrds_count = [w for w in tokens if w in stop_words]
    ratio = len(stop_wrds_count) / len(tokens)
    df['Ratio: Stop Words-Total Words'].values[i] = ratio

  #ADJECTIVES IN TWEET
  for i in range(0, len(df.index)):
    tokens = df['Tweet Tokens'][i]
    tokens = [w for w in tokens if not w in stop_words]
    tagged = nltk.pos_tag(tokens)

    adj_definitions = ['JJ', 'JJR', 'JJS ']
    count = 0

    for j in range(0, len(tagged)):
      if tagged[j][1] in adj_definitions:
        count += 1
    
    df['Count: Adjectives in Tweet'].values[i] = count

  # LOG OF TWEET WORD COUNT 
  for i in range(0, len(df.index)):           #In range: length of dataframe of tweets
    tokens = df['Tweet Tokens'][i]       #tokenize tweets 
    log_val = math.log(len(tokens))
    df['Log: Tweet word count'].values[i] = log_val


  # LOG LENGTH OF LONGEST WORD IN TWEET
  for i in range(0, len(df.index)):           #In range: length of dataframe of tweets
    tokens = df['Tweet Tokens'][i]   #tokenize tweets 
    longest_wrd = max(tokens, key=len)
    log_val =  math.log(len(longest_wrd))
    df['Log: Length of Longest Word in Tweet'].values[i] = log_val

  #LOG OF COUNT OF WORD WITH 5+ CHARACTERS
  for i in range(0, len(df.index)):           #In range: length of dataframe of tweets
    tokens = df['Tweet Tokens'][i]    #tokenize tweets
    count = 0
      
    if any(len(i) >= 5 for i in tokens) is True:

      for j in range(0, len(tokens)):
        if len(tokens[j]) >= 5:
          count = count +1

      log_val = math.log(count)
      df['Log: Count of Words with 5+ Characters'].values[i] = log_val
      
    else:
      df['Log: Count of Words with 5+ Characters'].values[i] = 0


  return(df)

In [352]:
train_file_location = "/content/drive/MyDrive/5290/5290.Data/train_text.csv"
train_label_location = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_labels.txt"

df_train = feature_extraction(train_file_location, train_label_location) #Create train dataframe

test_file_location = "/content/drive/MyDrive/5290/5290.Data/test_text.csv"
test_label_location ="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt"

df_test = feature_extraction(test_file_location, test_label_location)   #Create test dataframe

In [353]:
df_train

,TWEET,Tweet Tokens,Count: Words in + Lexicon,Count: Words in - Lexicon,Contain The word NO?,Count: Nouns,Ratio: Unique Words-Total Words,Ratio: Stop Words-Total Words,Count: Adjectives in Tweet,Log: Tweet word count,Log: Length of Longest Word in Tweet,Log: Count of Words with 5+ Characters,Labels
0,qt in the original draft of the 7th book remu...,"[qt, in, the, original, draft, of, the, 7th, b...",8.0,1.0,0.0,7.0,0.823529,0.352941,2.0,2.833213,3.135494,2.079442,1
1,alciato bee will invest 150 million in januar...,"[alciato, bee, will, invest, 150, million, in,...",10.0,4.0,0.0,5.0,0.950000,0.350000,1.0,2.995732,1.945910,2.197225,1
2,lit my mum kerry the louboutins wonder how m...,"[lit, my, mum, kerry, the, louboutins, wonder,...",5.0,5.0,0.0,7.0,0.933333,0.200000,1.0,2.708050,2.302585,1.945910,1
3,soul train oct 27 halloween special ft dot f...,"[soul, train, oct, 27, halloween, special, ft,...",15.0,6.0,0.0,11.0,1.000000,0.086957,4.0,3.135494,2.197225,2.302585,1
4,so disappointed in wwe summerslam want to see...,"[so, disappointed, in, wwe, summerslam, want, ...",6.0,5.0,0.0,5.0,1.000000,0.285714,2.0,2.639057,2.484907,1.098612,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24937,michael from good times is the 2nd greatest mi...,"[michael, from, good, times, is, the, 2nd, gre...",8.0,1.0,0.0,5.0,0.785714,0.285714,1.0,2.639057,2.079442,1.945910,1
24938,just think some misguided girl is in the vip ...,"[just, think, some, misguided, girl, is, in, t...",9.0,8.0,0.0,8.0,1.000000,0.391304,1.0,3.135494,2.708050,1.945910,0
24939,so amazing to have the beautiful lady gaga at...,"[so, amazing, to, have, the, beautiful, lady, ...",10.0,2.0,0.0,6.0,0.944444,0.444444,2.0,2.890372,2.197225,1.386294,1
24940,september has arrived which means apple new ...,"[september, has, arrived, which, means, apple,...",9.0,8.0,0.0,6.0,1.000000,0.388889,2.0,2.890372,2.197225,2.302585,1


In [354]:
df_test

,TWEET,Tweet Tokens,Count: Words in + Lexicon,Count: Words in - Lexicon,Contain The word NO?,Count: Nouns,Ratio: Unique Words-Total Words,Ratio: Stop Words-Total Words,Count: Adjectives in Tweet,Log: Tweet word count,Log: Length of Longest Word in Tweet,Log: Count of Words with 5+ Characters,Labels
0,think may be finally in with the in crowd ma...,"[think, may, be, finally, in, with, the, in, c...",4.0,3.0,0.0,3.0,0.909091,0.454545,0.0,2.397895,2.890372,1.609438,1
1,wow first hugo chavez and now fidel castro da...,"[wow, first, hugo, chavez, and, now, fidel, ca...",7.0,10.0,0.0,11.0,0.954545,0.272727,2.0,3.091042,1.945910,2.484907,0
2,twitter thankyouobama shows heartfelt gratitu...,"[twitter, thankyouobama, shows, heartfelt, gra...",3.0,0.0,0.0,4.0,1.000000,0.142857,1.0,1.945910,2.564949,1.791759,1
3,take away illegals and dead people and trump ...,"[take, away, illegals, and, dead, people, and,...",2.0,8.0,0.0,4.0,0.916667,0.250000,2.0,2.484907,2.079442,1.386294,0
4,onedirection harrystyles cute little dance,"[onedirection, harrystyles, cute, little, dance]",2.0,1.0,0.0,3.0,1.000000,0.000000,1.0,1.609438,2.484907,1.386294,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6342,for al the crying you do about how middle ame...,"[for, al, the, crying, you, do, about, how, mi...",4.0,13.0,0.0,4.0,1.000000,0.550000,2.0,2.995732,1.945910,1.945910,0
6343,not even catholic but pope francis is my dud...,"[not, even, catholic, but, pope, francis, is, ...",10.0,7.0,0.0,5.0,0.909091,0.500000,2.0,3.091042,2.302585,1.098612,1
6344,looks like flynn isn too pleased with me he ...,"[looks, like, flynn, isn, too, pleased, with, ...",5.0,5.0,0.0,3.0,0.750000,0.562500,2.0,2.772589,1.945910,1.791759,0
6345,trying to have conversation with my dad about...,"[trying, to, have, conversation, with, my, dad...",5.0,8.0,0.0,5.0,1.000000,0.470588,0.0,2.833213,2.564949,2.079442,0


In [356]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

X_train = df_train.drop(['Labels', 'TWEET', 'Tweet Tokens'],axis=1)
Y_train = df_train['Labels']
X_test = df_test.drop(['Labels','TWEET', 'Tweet Tokens'], axis=1)
Y_test = df_test['Labels']

X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values

In [360]:
logisticRegr = LogisticRegression(max_iter=1500)
logisticRegr.fit(X_train, Y_train)
predictions = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, Y_test)
print('Sklearn Logistic regression score: ', round(score *100, 2), '%')

Sklearn Logistic regression score:  61.48 %


In [361]:
cm = metrics.confusion_matrix(Y_test, predictions)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[1715 2257]
 [ 188 2187]]


In [362]:
precision = cm[0][0] / (cm[0][0] + cm[0][1])
recall = cm[0][0] / (cm[0][0] + cm[1][0])

F1 = 2 * (precision * recall) / (precision + recall)

print('F1: ', F1)

F1:  0.5838297872340427
